In [1]:
# Summary
################
# Image Acquire
# Color Plane Extraction & Gray Conversion
# Image Subtraction to fetch specific color
# Thresholding to convert to BW - CRITICAL
# Area/Region of Interest (ROI) - CRITICAL
    # Dilation/Erosion/Opening/Closing (combinations)
    # Noise Rmoval/Holes Filling
# Find the Region Properties
    # Label the Image
    # Regionprops (bounding Box) (y1,x1, y2,x2)
# Draw the rectangle over Original Image based on bbox

In [17]:
import cv2
vid = cv2.VideoCapture(0)
while True:
    ack, img = vid.read()
    if ack:
        # do the entire processing
        # ......
        cv2.imshow('Preview', img)  # depends on requirement
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows(); cv2.waitKey(1)
vid.release()

KeyboardInterrupt: 

In [16]:
# Live Image Capturing
import cv2, skimage as ski
vid = cv2.VideoCapture(0)
while True:
    ack, img = vid.read()
    if ack:
        th, red_bw = cv2.threshold(
            cv2.subtract(
                img[:,:,-3], cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ), 40, 255, cv2.THRESH_BINARY
        )
        strel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,20))
        red_bw = cv2.morphologyEx(
            red_bw, cv2.MORPH_CLOSE, strel, iterations=1
        )
        red_bw = ski.morphology.remove_small_objects(
            red_bw.astype(bool), 3000
        ).astype('uint8') * 255
        red_bw = ski.morphology.remove_small_holes(
            red_bw.astype(bool), 3000
        ).astype('uint8') * 255

        rps = ski.measure.regionprops(
            ski.measure.label(red_bw.astype(bool))
        )
        count = len(rps)
        img_copy = img.copy()
        cv2.putText(img_copy, str(count),(150,150),
                    cv2.FONT_HERSHEY_PLAIN, 10, (0,0,255),10
        )
        for rp in rps:
            y1,x1,y2,x2 = rp.bbox
            cv2.rectangle(img_copy, (x1,y1),(x2,y2),(0,0,255),thickness=5)

        cv2.imshow('Preview', img_copy)  # depends on requirement
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows(); cv2.waitKey(1)
vid.release()

In [19]:
# Face Detection - Viola-jones algo (Haarcascade features)
import cv2
fd = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
vid = cv2.VideoCapture(0)
while True:
    ack, img = vid.read()
    if ack:
        faces = fd.detectMultiScale(img,1.2,10)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),10)

        cv2.imshow('Preview', img)  # depends on requirement
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows(); cv2.waitKey(1)
vid.release()

In [ ]:
# Face Detection - Face Images Save to computer
import cv2
fd = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
vid = cv2.VideoCapture(0)
counter = 0
while True:
    ack, img = vid.read()
    if ack:
        faces = fd.detectMultiScale(img,1.2,10)
        if len(faces) == 1:
            counter += 1
            for x,y,w,h in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),10)
            face_img = img[y:y+h, x:x+w, :].copy()
            cv2.imwrite(
                r'faces/peeyush/face_{0}.png'.format(counter), 
                face_img
            )

        cv2.imshow('Preview', img)  # depends on requirement
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows(); cv2.waitKey(1)
vid.release()